<a href="https://colab.research.google.com/github/rakirs3333/DeepFakeDetection/blob/main/deepfake_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import glob
import numpy as np
import cv2
import copy

video_files =  glob.glob('/content/drive/My Drive/Celeb_fake_face_only/*.mp4')
video_files += glob.glob('/content/drive/My Drive/Celeb_real_face_only/*.mp4')
video_files += glob.glob('/content/drive/My Drive/DFDC_FAKE_Face_only_data/*.mp4')
video_files += glob.glob('/content/drive/My Drive/DFDC_REAL_Face_only_data/*.mp4')
video_files += glob.glob('/content/drive/My Drive/FF_Face_only_data/*.mp4')

frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

In [ ]:
def frame_extract(path):
    video = cv2.VideoCapture(path)
    frame_present=1
    while frame_present:
        # read frame by frame
        frame_present, frame = video.read()
        if frame_present:
            yield frame

!pip3 install face_recognition
!mkdir '/content/drive/My Drive/Facial_data'

     |████████████████████████████████| 100.1 MB 19 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=9240055ae944935a4cd708b5c554490f35e0f3bdbacd9155f750cfc8f48c622e
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models
mkdir: cannot create directory ‘/content/drive/My Drive/Facial_data’: File exists


In [ ]:
!pip3 install face_recognition
import os
import numpy as np
import cv2
import face_recognition
from tqdm.autonotebook import tqdm

def create_face_videos(path_list,out_dir):
    for path in tqdm(path_list):
      out_path = os.path.join(out_dir,path.split('/')[-1])
      file_exists = glob.glob(out_path)
      if(len(file_exists) != 0):
        print("File Already exists: " , out_path)
        continue
      frames = []
      out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))#fourcc is 4 char code(encoding format)
      for idx,frame in enumerate(frame_extract(path)):
        if(idx <= 150):
          frames.append(frame)
          if(len(frames) == 4):
            faces = face_recognition.batch_face_locations(frames)
            for i,face in enumerate(faces):
              if(len(face) != 0):
                top,right,bottom,left = face[0]
              try:
                out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
              except:
                pass
            frames = []
      out.release()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


In [ ]:
create_face_videos(video_files,'/content/drive/My Drive/Facial_data')

  0%|          | 0/16 [00:00<?, ?it/s]